In [1]:
!pip install git+https://github.com/afnan47/cuda.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-8fyefoap
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-8fyefoap
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=71ca03e68f91ffcd217d56af228e2b35d9dd6425c5106e5a8b22c4f9bcef7353
  Stored in directory: /tmp/pip-ephem-wheel-cache-uh6b9wxf/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [3]:
%%cu
#include <iostream>
int main(){
  std::cout << "Hello World\n";
  return 0;
}

Hello World



In [5]:
%%cu
// HPC 4 A
#include<iostream>
using namespace std;

__global__
void add(int* a, int* b,int* c,int size){
    int tid = blockIdx.x*blockDim.x + threadIdx.x;
    if(tid<size){
        c[tid] = a[tid] + b[tid];
    }
}
void initialize(int* vector,int size){
    for(int i=0;i<size;i++){
        vector[i] = rand()%10;
    }
}
void print(int* vector,int size){
    for(int i=0;i<size;i++){
        cout<<vector[i]<<" ";
    }
}

int main(){
    int n=4;
    int* a, * b, * c;
    
    int vectorSize = n;

    size_t vectorBytes = vectorSize* sizeof(int);

    a = new int[vectorSize];
    b = new int[vectorSize];
    c = new int[vectorSize];

    initialize(a,n);
    initialize(b,n);

    cout<<"\nVector 1 : ";
    print(a,n);
    cout<<"\nVector 2 : ";
    print(b,n);

    int *x, *y, *z;

    cudaMalloc(&x,vectorBytes);
    cudaMalloc(&y,vectorBytes);
    cudaMalloc(&z,vectorBytes);

    cudaMemcpy(x,a,vectorBytes,cudaMemcpyHostToDevice);
    cudaMemcpy(y,b,vectorBytes,cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocksPerGrid = (n+ threadsPerBlock -1)/threadsPerBlock;

    add<<<blocksPerGrid,threadsPerBlock>>>(x,y,z,n);

    cudaMemcpy(c,z,vectorBytes,cudaMemcpyDeviceToHost);

    cout<<"\nAddition : ";
    print(c,n);

    delete[] a;
    delete[] b;
    delete[] c;

    cudaFree(x);
    cudaFree(y);
    cudaFree(z);


    return(0);
    
}


Vector 1 : 3 6 7 5 
Vector 2 : 3 5 6 2 
Addition : 6 11 13 7 


In [6]:
%%cu
//HPC 4 B
#include<iostream>
using namespace std;

__global__ void multiply(int* a,int* b,int*c ,int size){
    int row=blockIdx.y*blockDim.y + threadIdx.y;
    int col=blockIdx.x*blockDim.x + threadIdx.x;

    if(row<size && col<size){
        int sum=0;
        for(int i=0;i<size;i++){
            sum+=a[row*size + i]* b[size*i + col];
        }
        c[row*size + col] = sum;
    }
}

void initialize(int* matrix,int size){
    for(int i=0;i<size*size;i++){
        matrix[i] = rand()%10;
    }
}

void print(int* matrix,int size){
    for(int row=0;row < size;row++){
        for(int col=0;col<size;col++){
            cout<<matrix[row*size + col]<<" ";
        }
        cout<<"\n";
    }
}


int main(){
    int *a,*b,*c;

    int n=2;

    int blockSize=16;
    int matrixSize=n*n;
    size_t matrixBytes = matrixSize * sizeof(int);

    a = new int[matrixSize];
    b = new int[matrixSize];
    c = new int[matrixSize];

    initialize(a,n);
    initialize(b,n);

    cout<<"\nMatrix A \n";
    print(a,n);
    cout<<"\nMatrix B \n";
    print(b,n);

    int *x,*y,*z;

    cudaMalloc(&x,matrixBytes);
    cudaMalloc(&y,matrixBytes);
    cudaMalloc(&z,matrixBytes);

    cudaMemcpy(x,a,matrixBytes,cudaMemcpyHostToDevice);
    cudaMemcpy(y,b ,matrixBytes,cudaMemcpyHostToDevice);

    int THREADS=2;
    int BLOCKS = n/THREADS;

    dim3 threads(THREADS,THREADS);
    dim3 blocks(BLOCKS,BLOCKS);

    multiply<<<blocks,threads>>>(x,y,z,n);

    cudaMemcpy(c,z,matrixBytes,cudaMemcpyDeviceToHost);

    cout<<"\nMatrix Multiplication \n";
    print(c,n);

    delete[] a;
    delete[] b;
    delete[] c;

    cudaFree(x);
    cudaFree(y);
    cudaFree(z);

    return(0);


}


Matrix A 
3 6 
7 5 

Matrix B 
3 5 
6 2 

Matrix Multiplication 
45 27 
51 45 

